In [ ]:
# 회귀
# 출력의 개수를 1개로
# 손실함수를 MSE나 기타 등등..
# 데이터셋 과 데이터 로드를 커스텀하게 정의해서 사용
# 나머지는 동일한 패턴으로 ... 학습 평가

In [ ]:
# from sklearn.datasets import load_boston

In [ ]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torch

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

class BostonDataSet(Dataset):
    def __init__(self,X,y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1,1) # 일대일대응으로 값을 넣기 위해? ^
    def __len__(self):
        return len(self.X)
    def __getitem__(self,idx):
        return self.X[idx], self.y[idx]

In [ ]:
X_dataset = BostonDataSet(data, target)  # 데이터를 X, y 를 한쌍으로 묶어
X_train_loader = DataLoader(X_dataset ,batch_size=32,shuffle=True) # 배치단위로 묶어서 데이터 로드
                                                                   # 중복가능성이 있을듯 ^

In [ ]:
# 회귀 모델 정의
import torch.nn as nn
class BostonRegression(nn.Module): # nn.Module에서 상속을 받는다.
    def __init__(self, input_dim):
        super(BostonRegression,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(), # 기울기 사라짐, 폭파, nan 등 학습과정에서의 오류발생을 막는다.
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )
    def forward(self, x):
        return self.model(x)

In [ ]:
from torch.optim import Adam
model = BostonRegression(data.shape[1])
criterion = nn.MSELoss()
optim = Adam(model.parameters(), lr = 1e-3) # 옵티마이저를 통해 파라미터를 업데이트
                                            # 모델에 사용하고 있는 파라미터 지정 : model.parameters()

In [21]:
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
epochs = 100
# 학습루프
for epoch in range(epochs):
    tqdm_obj = tqdm(X_train_loader,desc=f'epoch : {epoch+1}/{epochs}')
    loss_lists = 0
    for data, label in tqdm_obj:
        optim.zero_grad()
        preds = model(data.to(device))
        loss = criterion(preds, label.to(device))
        loss_lists += loss.item()
        loss.backward()
        optim.step()    
        tqdm_obj.set_postfix({'loss' : f'{loss.item():.4f}'})
    avg_loss = loss_lists / len(X_train_loader)
    print(f'epoch : {epoch+1} : avg loss : {avg_loss:.4f}')
        

torch.save(model.state_dict(), 'bostonRegression.pth') 

epoch : 1/100: 100%|██████████| 16/16 [00:00<00:00, 352.10it/s, loss=7.7949]


epoch : 1 : avg loss : 17.2570


epoch : 2/100: 100%|██████████| 16/16 [00:00<00:00, 348.12it/s, loss=15.7468]


epoch : 2 : avg loss : 16.1514


epoch : 3/100: 100%|██████████| 16/16 [00:00<00:00, 385.95it/s, loss=18.5483]


...

epoch : 98 : avg loss : 12.1474


epoch : 99/100: 100%|██████████| 16/16 [00:00<00:00, 390.64it/s, loss=9.6713]


epoch : 99 : avg loss : 13.4169


epoch : 100/100: 100%|██████████| 16/16 [00:00<00:00, 344.37it/s, loss=8.2769]


epoch : 100 : avg loss : 14.0992


In [ ]:
# 학습 과정중 loss값이 많이왔다 갔다 한다. 학습이 불안정하다고 할 수 있다.

In [24]:
from sklearn.metrics import r2_score
# 평가
model.load_state_dict(torch.load('bostonRegression.pth',map_location=device,weights_only=True))
# 예측
# 평가 루프
model.eval()  # 평가 모드로 전환 (dropout, batchnorm 등 비활성화)
total_mse = 0
r2_scores = 0

criterion = nn.MSELoss()
with torch.no_grad():  # 그래디언트 계산 비활성화
    for data, label in tqdm(X_train_loader, desc="Evaluating"):
        data, label = data.to(device), label.to(device)
        preds = model(data)
        r2_scores += r2_score(label.detach().numpy(), preds.detach().numpy()) # 결정계수 조회
        mse = criterion(preds, label)        
        total_mse += mse.item()
print(f"Test Loss: {total_mse / len(X_train_loader)} r2_score : {r2_scores/len(X_train_loader)}")

Evaluating: 100%|██████████| 16/16 [00:00<00:00, 800.44it/s]

Test Loss: 12.016757518053055 r2_score : 0.8413007035851479
